In [1]:
import pandas as pd
import numpy as np
from googleapiclient.discovery import build

   + Active my credentials for the YouTube v3 api

In [2]:
api_key = open("api_key.txt","r")
api_key = api_key.read()
youtube = build('youtube', 'v3', developerKey=api_key)

   + Function to get the channel ID only from the User input using youtube.search.list()

In [9]:
def get_channel_ids(user_input):
    try:
        request = youtube.search().list(
            part="snippet",
            order="relevance",
            maxResults=5,
            q=user_input,
        )
        response = request.execute()

        channel_ids = []
        for item in response['items']:
            channel_id = item['snippet']['channelId']
            if channel_id not in channel_ids:  
                channel_ids.append(channel_id)

        if channel_ids:
            return channel_ids
        else:
            print("Sorry, I can't find your topic in my recommendations. Let's try another topic :).")
            return None
    except Exception as e:
        print("An error occurred:", str(e))
        return None

user_input = input("Type a topic for your channel: ")
channel_ids = get_channel_ids(user_input)


Type a topic for your channel: football boots


In [10]:
# Display the channels 
channel_ids

['UCwozCpFp9g9x0wAzuFh0hwQ',
 'UC5SQGzkWyQSW_fe-URgq7xw',
 'UCUU3lMXc6iDrQw4eZen8COQ']

   + Function to get the tags and category ID from youtube.video.list()

In [11]:
def get_video_ids(channel_ids):
    video_ids = []
    for channel_id in channel_ids:
        try:
            request = youtube.search().list(
                part="snippet",
                channelId=channel_id,
                maxResults=1,
                q=user_input,
                type="video"
            )
            response = request.execute()

            video_id = response['items'][0]['id']['videoId']
            video_ids.append(video_id)
        except Exception as e:
            print("An error occurred:", str(e))

    return video_ids

video_ids = get_video_ids(channel_ids)


def get_tags_channel(video_ids):
    database = []
    for video_id in video_ids:
        try:
            request = youtube.videos().list(
                part="snippet",
                id=video_id
            )
            response = request.execute()

            items = response['items']
            for item in items:
                video_data = [
                    item['snippet'].get('channelId'),
                    item['snippet'].get('channelTitle'),
                    item['snippet'].get('categoryId'),
                    item['snippet'].get('title'),
                    item['snippet'].get('tags'),
                    item['snippet'].get('description')
                ]

                channel_url = item['snippet'].get('channelId')

                request_1 = youtube.channels().list(
                    part="snippet",
                    id=channel_url
                )
                response_1 = request_1.execute()

                channel_data = response_1['items'][0]['snippet']
                video_data.append(channel_data.get('customUrl'))

                database.append(video_data)
        except Exception as e:
            print("An error occurred:", str(e))

    return database

database = get_tags_channel(video_ids)

df_channel = pd.DataFrame(database, columns=['channel_id', 'channel_title', 'category_id', 'title', 'tags', 'description', 'channel_url'])


In [12]:
top_5_channel = df_channel.drop(columns=['channel_id', 'category_id','title', 'tags', 'description'])
top_5_channel

,channel_title,channel_url
0,FC Motivate,@_fcmotivate
1,Unisport,@unisport
2,Soccer Reviews For You,@soccerreviewsforyou


In [14]:
video_reccomendation = df_channel.drop(columns=['channel_id', 'channel_title', 'category_id', 'tags', 'description', 'channel_url'])
video_reccomendation

,title
0,The Story Behind Jack Grealish’s Ripped Footba...
1,5 mistakes to AVOID when buying new boots
2,The NEW and CONFUSING Adidas X CrazyFast footb...
